<a href="https://colab.research.google.com/github/mukulsn/Machine-Learning/blob/main/Spacy/Spacy_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-12-03 09:23:33--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-12-03 09:23:33 (113 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [ ]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2023-12-03 09:23:34--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.148.207, 142.251.171.207, 209.85.200.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.148.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2023-12-03 09:23:35 (151 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:
# Turn .csv files into pandas DataFrame's
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
# splitting training data into train and validation dataset
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df['text'].to_numpy(),
                                                                            train_df['target'].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)
train_df = pd.DataFrame({'text':train_sentences,'target':train_labels})
val_df = pd.DataFrame({'text':val_sentences,'target':val_labels})

# Text Classification Using SPacy
Resource :
https://catherinebreslin.medium.com/text-classification-with-spacy-3-0-d945e2e8fc44

https://medium.com/@johnidouglasmarangon/building-a-text-classification-model-with-spacy-3-x-57e59fa50547

Label manually:
1. Prodigy
2. Label Studio

* `0` = Not a real diaster Tweet
* `1` = Real diaster Tweet

In [ ]:

import re
from spacy.lang.pt.stop_words import STOP_WORDS
import string

REGX_USERNAME = r"@[A-Za-z0-9$-_@.&+]+"
REGX_URL = r"https?://[A-Za-z0-9./]+"

def preprocessing(text):
  text = text.lower()

  text = re.sub(REGX_USERNAME, ' ', text)
  text = re.sub(REGX_URL, ' ', text)

  # emojis = {
  #     ':)': 'emocaopositiva',
  #     ':(': 'emocaonegativa'
  # }

  # for e in emojis:
  #   text = text.replace(e, emojis[e])

  tokens = [token.text for token in nlp(text)]

  tokens = [t for t in tokens if
              t not in STOP_WORDS and
              t not in string.punctuation and
              len(t) > 3]

  tokens = [t for t in tokens if not t.isdigit()]

  return " ".join(tokens)

In [ ]:
train_df['text'] = train_df['text'].apply(preprocessing)
val_df['text'] = val_df['text'].apply(preprocessing)
test_df['text'] = test_df['text'].apply(preprocessing)

In [ ]:
# preparing data for spacy

train_df = list(train_df.itertuples(index=False, name=None))
val_df = list(val_df.itertuples(index=False, name=None))
test_df = list(test_df.drop(columns=['id','keyword','location']).itertuples(index=False, name=None))

In [ ]:
from spacy.tokens import DocBin
import spacy

nlp = spacy.blank('en')
def convert(data, outfile):
  db = spacy.tokens.DocBin()
  docs = []
  for doc, label in nlp.pipe(data, as_tuples=True):
    doc.cats['POS'] = label == 1
    doc.cats['NEG'] = label == 0
    db.add(doc)
  db.to_disk(outfile)

convert(train_df,"./train.spacy")
convert(val_df,"./dev.spacy")
# convert(test_df,"./test.spacy")


# downloading webpage of medium

In [ ]:
!apt-get install wkhtmltopdf

In [ ]:
import pdfkit
pdfkit.from_url('https://catherinebreslin.medium.com/text-classification-with-spacy-3-0-d945e2e8fc44', 'spacy classification2.pdf')

True

In [ ]:
!cp "/content/spacy classification.pdf" "/content/drive/MyDrive/Colab Notebooks/ML projects/Tensorflow projects/practice notebooks/NLP/"

In [ ]:
!cp "/content/spacy classification2.pdf" "/content/drive/MyDrive/Colab Notebooks/ML projects/Tensorflow projects/practice notebooks/NLP/"

# spacy configuration


In [ ]:
!python -m spacy download en_core_web_lg

2023-12-03 11:00:41.560113: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-03 11:00:41.560169: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-03 11:00:41.560193: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-03 11:00:42.518826: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
!python -m spacy init config --lang en --pipeline textcat --optimize efficiency --force config.cfg

2023-12-03 09:41:16.436299: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-03 09:41:16.436358: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-03 09:41:16.436386: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-03 09:41:17.630459: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case

In [ ]:
!python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy --output model --verbose

2023-12-03 10:12:49.686687: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-03 10:12:49.686761: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-03 10:12:49.686837: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-03 10:12:52.041207: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[2023-12-03 10:12:54,362] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
ℹ Saving to output directory: model
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-12-03 10:12:54,

In [ ]:
# evaluation

!python -m spacy evaluate ./model/model-best/ ./dev.spacy

2023-12-03 10:13:22.391628: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-03 10:13:22.391693: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-03 10:13:22.391722: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-03 10:13:23.325543: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Using CPU

================================== Results ==================================

TOK                 100.00
TEXTCAT (macro F)   77.06 
SPEED               211965


=========================== Textcat F (per label) =============

In [ ]:
# test pipeline
nlp = spacy.load('./model/model-best')

doc = nlp(preprocessing('ther is fire in forest'))
doc.cats

{'POS': 0.6594340205192566, 'NEG': 0.3405659794807434}

In [ ]:
preprocessing('hello is this a disaster tweet australia')

'hello this disaster tweet australia'

In [ ]:
!python -m spacy init fill-config ./base_config.cfg ./config.cfg

2023-12-03 10:50:20.996084: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-03 10:50:20.996137: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-03 10:50:20.996164: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-03 10:50:21.961613: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy --output model --verbose

2023-12-03 11:01:30.667499: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-03 11:01:30.667573: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-03 11:01:30.667611: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-03 11:01:32.043356: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[2023-12-03 11:01:34,260] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
ℹ Saving to output directory: model
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-12-03 11:01:34,

In [ ]:
# evaluation

!python -m spacy evaluate ./model/model-best/ ./dev.spacy

2023-12-03 11:23:26.650045: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-03 11:23:26.650111: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-03 11:23:26.650142: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-03 11:23:27.666730: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Using CPU

================================== Results ==================================

TOK                 100.00
TEXTCAT (macro F)   77.05 
SPEED               3002  


=========================== Textcat F (per label) =============

In [ ]:
# test pipeline
nlp = spacy.load('./model/model-best')

doc = nlp(preprocessing('there is a disaster tweet'))
doc.cats

{'POS': 0.4325772821903229, 'NEG': 0.5674226880073547}

In [ ]:
bool(0)

False